In [1]:
"""
GitHub - code referenced from  https://github.com/stianyu/BCI_Competition_III_IVa.git

mne documentation for CSP filtering and LDA implementation
https://mne.tools/dev/auto_examples/decoding/plot_decoding_csp_eeg.html

GitHub issue raised for not having montage as the plot_patterns are not working
https://github.com/mne-tools/mne-python/issues/4835

Dataset description   http://www.bbci.de/competition/iv/desc_1.html

In this dataset file c, d, e have the artificial data and a, b, f, g have the original data so the results can be different
files

"""




import numpy as np
from scipy.io import loadmat
from scipy.signal import butter,filtfilt
import scipy.signal as sg
import plotly as py
import plotly.graph_objs as go
from plotly import tools
from scipy.fftpack import fft, ifft
from plotly.subplots import make_subplots

py.offline.init_notebook_mode(connected=True)

%matplotlib inline
!pip install --upgrade pip
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
! pip install -q scikit-plot
import scikitplot as skplt
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

ModuleNotFoundError: No module named 'seaborn'

In [29]:
reg_evaluation = pd.DataFrame({'Model': [],
                           'Details':[],
                           'RMSE(train)':[],
                           'R-squared (train)':[],
                           'Adj R-squared (train)':[],
                           'MAE (train)':[],
                           'RMSE (test)':[],
                           'R-squared (test)':[],
                           'Adj R-squared (test)':[],
                           'MAE(test)':[],
                           '10-Fold Cross Validation':[]})

reg_evaluation2 = pd.DataFrame({'Model': [],
                           'Test':[],
                           '1':[],
                           '2':[],
                           '3':[],
                           '4':[],
                           '5':[],
                           '6':[],
                           '7':[],
                           '8':[],
                           '9':[],
                           '10':[],
                           'Mean':[]})

classification_evaluation = pd.DataFrame({'Model': [],
                           'Accuracy(train)':[],
                           'Precision(train)':[],
                           'Recall(train)':[],
                           'F1_score(train)':[],
                           'Accuracy(test)':[],
                           'Precision(test)':[],
                           'Recalll(test)':[],
                           'F1_score(test)':[]})

classification_evaluation2 = pd.DataFrame({'Model': [],
                           'Test':[],
                           '1':[],
                           '2':[],
                           '3':[],
                           '4':[],
                           '5':[],
                           '6':[],
                           '7':[],
                           '8':[],
                           '9':[],
                           '10':[],
                           'Mean':[]})

In [ ]:
file1 = r'G:\Machine-Learning_BCI\Datasets\MAT_Format\BCICIV_1calib_1000Hz_mat\BCICIV_calib_ds1a_1000Hz'
F1 = loadmat(file1)

In [ ]:
type(F1)

In [ ]:
F1.keys()

In [ ]:
td = {}
F1['cnt']

In [ ]:
td['data'] = F1['cnt'].T

In [ ]:
F1['nfo']['fs'][True][0]

In [ ]:
F1['nfo']['classes'][True][0]

In [ ]:
F1['nfo']['clab'][True][0]

In [ ]:
F1['nfo']['xpos'][True][0]

In [ ]:
F1['nfo']['ypos'][True][0]

In [ ]:
F1['mrk']['pos'][True][0] 

In [ ]:
type(F1['mrk']['y'][True][0])

In [ ]:
type(F1['nfo']['classes'][True][0])

In [ ]:
import scipy.io as sio

fp = r'G:\Machine-Learning_BCI\Datasets\MAT_Format\BCICIV_1calib_1000Hz_mat\BCICIV_calib_ds1a_1000Hz'

mat = sio.loadmat(fp, squeeze_me=True)


In [ ]:
mat.keys()

In [ ]:
mat['mrk']['y'][True][0].shape

In [ ]:
t = {}
t['data'] = mat['cnt'].T

In [ ]:
t['data'].shape

In [ ]:
mat['cnt'].shape

In [ ]:
t['freq'] = mat['nfo']['fs'][True][0]
t['freq']

In [ ]:
t['ch_names'] = mat['nfo']['clab'][True][0]
t['ch_names']

In [ ]:
t['elctrode_x'] = mat['nfo']['xpos'][True][0]
t['elctrode_x']

In [ ]:
t['electrode_y'] = mat['nfo']['ypos'][True][0]
t['electrode_y']

In [ ]:
t['cue'] = mat['mrk']['pos'][True][0] 
t['cue']

In [ ]:
# t['labels'] = np.nan_to_num(mat['mrk']['y'][True][0]).astype(int)
t['labels'] = mat['mrk']['y'][True][0]
t['labels'].size

In [ ]:
np.where(t['labels'] == -1)[0]

In [ ]:
np.where(t['labels'] == 1)[0]

In [ ]:
testing_trials = np.where(t['labels'] == True)[0]
testing_trials

In [ ]:
t.keys()

In [ ]:
t

In [ ]:
t['classes'] = F1['nfo']['classes'][True][0]
t['classes']

In [ ]:
mat['nfo']

In [ ]:
t['n_trials'] = np.where(t['labels'] == 1)[0]
t['n_trials'].shape

In [ ]:
t.keys()

In [ ]:
out = np.array(t)
out

In [2]:
#preprocessors

import numpy as np
import scipy.io as sio

import mne
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.decoding import CSP




def inputmat(fp):
    """load .mat file and return m as a dict"""
    mat = sio.loadmat(fp, squeeze_me=True)
    td = {}  # create a dict

    # Numpy array of size channel_num * points.
    td['data'] = mat['cnt'].T  # data
    td['freq'] = mat['nfo']['fs'][True][0]  # Sampling frequency

    # channel names are necessary information for creating a rawArray.
    td['ch_names'] = mat['nfo']['clab'][True][0]

    # Position of channels
    td['electrode_x'] = mat['nfo']['xpos'][True][0]
    td['electrode_y'] = mat['nfo']['ypos'][True][0]

    #make trials by finding trials and its data
    td['cue'] = mat['mrk']['pos'][True][0] #time of cue
    td['labels'] = mat['mrk']['y'][True][0] #labels of the data
    return td


""" do we really need number of trials and also what is the meaning of the line in 
events[:, 0] and events[:, 2]"""



def creatEventsArray(fp):
    """Create events array. The second column default to zero."""
    td = inputmat(fp)
    events = np.zeros((td['labels'].size, 3), int) #here we have made a matrix type array of the size of label.size*3
#     print(events)
    events[:, 0] = td['cue']  # The first column is the sample number of the event.
#     print(events[:, 0])
    events[:, 2] = td['labels']  # The third column is the new event value.
#     print(events[:, 2])
    return events, td['labels']


def creatRawArray(fp):
    """Create a mne.io.RawArray object, data: array, shape (n_channels, n_times)"""
    td = inputmat(fp)
    ch_names = td['ch_names'].tolist()
    info = mne.create_info(ch_names, td['freq'], 'eeg')  # Create info for raw
    raw = mne.io.RawArray(td['data'], info, first_samp=0, copy='auto', verbose=None)
    return raw


In [3]:
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline

from mne.decoding import CSP
from mne.channels import read_layout

In [4]:
"""The files c, d, e are the artificial data so we can test out our labels in different ways in them"""


file1 = r'G:\Machine-Learning_BCI\Datasets\MAT_Format\BCICIV_1calib_1000Hz_mat\BCICIV_calib_ds1a_1000Hz'
# F1 = loadmat(file1)

file2 = r'G:\Machine-Learning_BCI\Datasets\MAT_Format\BCICIV_1calib_1000Hz_mat\BCICIV_calib_ds1b_1000Hz'
# F2 = loadmat(file2)

# file3 = r'G:\Machine-Learning_BCI\Datasets\MAT_Format\BCICIV_1calib_1000Hz_mat\BCICIV_calib_ds1c_1000Hz'
# F3 = loadmat(file3)

# file4 = r'G:\Machine-Learning_BCI\Datasets\MAT_Format\BCICIV_1calib_1000Hz_mat\BCICIV_calib_ds1d_1000Hz'
# F4 = loadmat(file4)

# file5 = r'G:\Machine-Learning_BCI\Datasets\MAT_Format\BCICIV_1calib_1000Hz_mat\BCICIV_calib_ds1e_1000Hz'
# F5 = loadmat(file5)

file6 = r'G:\Machine-Learning_BCI\Datasets\MAT_Format\BCICIV_1calib_1000Hz_mat\BCICIV_calib_ds1f_1000Hz'
# F6 = loadmat(file6)

file7 = r'G:\Machine-Learning_BCI\Datasets\MAT_Format\BCICIV_1calib_1000Hz_mat\BCICIV_calib_ds1g_1000Hz'
# F7 = loadmat(file7)



"""making the file dictionary and channel dictionary for easy pickup"""
fp = {
    'ds1a': file1,
    'ds1b': file2,
#     'ds1c': file3, #these three are artificial datatsets
#     'ds1d': file4,
#     'ds1e': file5,
    'ds1f': file6,
    'ds1g': file7,
}

# pick_chan = {
#     'ds1a': ['C3', 'Cz', 'C5'],
#     'ds1b': ['C3', 'Cz', 'C5'],
#     'ds1c': ['C3', 'Cz', 'C5'],  
#     'ds1d': ['C3', 'Cz', 'C5'],
#     'ds1e': ['C3', 'Cz', 'C5'],
#     'ds1f': ['C3', 'Cz', 'C5'],
#     'ds1g': ['C3', 'Cz', 'C5'],
# }

low_freq, high_freq = 7., 30.
tmin, tmax = 0., 3.5

# event_id
# event_id = {'right': 1, 'foot': 2}
#changing eventid from right to left
event_id = {'left': -1, 'foot': 1}

In [22]:
raw = creatRawArray(file1)
events, labels = creatEventsArray(file1)
    
# strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))

#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)


# Apply band-pass filter
raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')

#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

# event_train = eventsTrain(fp[f])
epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                    verbose=False)#, picks=picks)

epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
labels = epochs.events[:, -1] - 2

# Define a monte-carlo cross-validation generator (reduce variance):
scores = []
epochs_data = epochs.get_data()
epochs_data_train = epochs_train.get_data()

#splitting the data
X_train, X_test, y_train, y_test = train_test_split(epochs_data_train, labels, test_size=0.2, random_state=42)


cv = ShuffleSplit(5, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data_train)

print(epochs_data_train.shape)
print(cv)
print(cv_split)

Creating RawArray with float64 data, n_channels=59, n_times=1905940
    Range : 0 ... 1905939 =      0.000 ...  1905.939 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 7 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (1.651 sec)

(200, 59, 1001)
ShuffleSplit(n_splits=5, random_state=42, test_size=0.2, train_size=None)
<generator object BaseShuffleSplit.split at 0x00000216890C8D60>


In [24]:
def adjustedR2(r2,n,k=3):
    return r2-(k-1)/(n-k)*(1-r2)

In [36]:
from sklearn import metrics


# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([('CSP', csp), ('LDA', lda)])



clf.fit(X_train, y_train)
features = 3
pred = clf.predict(X_test)
rmse_train = float(format(np.sqrt(metrics.mean_squared_error(clf.predict(X_train), y_train)),'.3f'))
r2_train = float(format(clf.score(X_train, y_train),'.3f'))
# ar2_train = float(format(adjustedR2(clf.score(X_train, y_train),X_train.shape[0],3),'.3f'))
mae_train=float(format((metrics.mean_absolute_error(clf.predict(X_train), y_train)),'.3f'))

rmse_test = float(format(np.sqrt(metrics.mean_squared_error(clf.predict(X_test), y_test)),'.3f'))
r2_test = float(format(clf.score(X_test, y_test),'.3f'))
# ar2_test = float(format(adjustedR2(clf.score(X_test, y_test),X_test.shape[0],3),'.3f'))
mae_test=float(format((metrics.mean_absolute_error(clf.predict(X_test), y_test)),'.3f'))












# scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

cv = float(format(cross_val_score(clf,X_train, y_train,cv=10).mean(),'.3f'))
r = reg_evaluation.shape[0]
reg_evaluation.loc[r] = ['SVR','All features',rmse_train,r2_train,ar2_train,mae_train,rmse_test,r2_test,ar2_test,mae_test,cv]


# # Print the predicted and actual value for the test set
# MLR_y_test_prediction= clf.predict(X_test)
# np.savetxt('SVR_test_prediction.csv', MLR_y_test_prediction, delimiter=',', fmt='%s')
# np.savetxt('SVR_test_actual.csv', y_test, delimiter=',', fmt='%s')

# # Print the predicted and actual value for the traing set
# MLR_y_train_prediction= clf.predict(X_train)
# np.savetxt('SVR_train_prediction.csv', MLR_y_train_prediction, delimiter=',', fmt='%s')
# np.savetxt('SVR_train_actual.csv', y_train, delimiter=',', fmt='%s')

# X_standardized = scaler.transform(X)
# MLR_y_pred_entire_data = clf.predict(X_standardized)
# np.savetxt('SVR_entire_prediction.csv', MLR_y_pred_entire_data, delimiter=',', fmt='%s')
# np.savetxt('SVR_entire_actual.csv', y, delimiter=',', fmt='%s')

# import matplotlib.pyplot as plt
# plt.plot(pred, y_test,  'ro')
# plt.ylabel('Predicted data')
# plt.xlabel('Actual data')
# plt.show()

# print(MLR_y_test_prediction)
# print(MLR_y_train_prediction)
# print(intact)


# Printing the results
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

# plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

Computing rank from data with rank=None
    Using tolerance 2.3e+03 (2.2e-16 eps * 59 dim * 1.8e+17  max singular value)
    Estimated rank (mag): 59
    MAG: rank 59 computed from 59 data channels with 0 projectors
Reducing data rank from 59 -> 59
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.2e+03 (2.2e-16 eps * 59 dim * 1.7e+17  max singular value)
    Estimated rank (mag): 59
    MAG: rank 59 computed from 59 data channels with 0 projectors
Reducing data rank from 59 -> 59
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.2e+03 (2.2e-16 eps * 59 dim * 1.7e+17  max singular value)
    Estimated rank (mag): 59
    MAG: rank 59 computed from 59 data channels with 0 projectors
Reducing data rank from 59 -> 59
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 2.1e+03 (2.2e-16 eps * 59 dim * 1.6e+17  max singular value)


C:\Users\rahul\AppData\Local\Continuum\anaconda3\envs\mne\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning:

Mean of empty slice.

C:\Users\rahul\AppData\Local\Continuum\anaconda3\envs\mne\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars



In [ ]:
"""code from the mne documentation
https://mne.tools/dev/auto_examples/decoding/plot_decoding_csp_eeg.html"""


for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    lda = LinearDiscriminantAnalysis()
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('LDA', lda)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
"""code from the mne documentation
https://mne.tools/dev/auto_examples/decoding/plot_decoding_csp_eeg.html"""

#this is the SVM classifier
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from mne.decoding import Scaler

for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    
    #shuffling the data
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    SVM_C = svm.SVC(kernel='rbf', degree=100)
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('SVM_C', SVM_C)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
"""code from the mne documentation
https://mne.tools/dev/auto_examples/decoding/plot_decoding_csp_eeg.html"""

#this is the Random Forest classifier
from sklearn.ensemble import RandomForestClassifier


for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    
    #shuffling the data
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    RFC = RandomForestClassifier()
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('RFC', RFC)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
"""code from the mne documentation
https://mne.tools/dev/auto_examples/decoding/plot_decoding_csp_eeg.html"""

#this is the MLP neural net classifier
from sklearn.neural_network import MLPClassifier


for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    
    #shuffling the data
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    MLP_C = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(16, 16), random_state=42)
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('MLP_C', MLP_C)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
"""code from the mne documentation
https://mne.tools/dev/auto_examples/decoding/plot_decoding_csp_eeg.html"""

#this is the Random Forest classifier
from sklearn.ensemble import RandomForestClassifier


for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    
    #shuffling the data
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    RFC = RandomForestClassifier()
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('RFC', RFC)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
"""code reference from the mne documentation
https://www.researchgate.net/publication/321726030_The_Performance_Analysis_of_K-Nearest_Neighbors_K-NN_Algorithm_for_Motor_Imagery_Classification_Based_on_EEG_Signal

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

"""

#this is the Knearest neighbour classifier
from sklearn.neighbors import KNeighborsClassifier


for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    
    #shuffling the data
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    KNN_C = KNeighborsClassifier(n_neighbors=15, algorithm='auto', n_jobs=1, metric='chebyshev')
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('KNN_C', KNN_C)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
"""code reference from the mne documentation

"""

#this is the Decision Tree classifier
from sklearn.tree import DecisionTreeClassifier


for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    
    #shuffling the data
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    DTC = DecisionTreeClassifier(random_state=42, criterion='gini')
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('DTC', DTC)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
"""code reference from the mne documentation

"""

#this is the Logistic regression
from sklearn.linear_model import LogisticRegression


for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    
    #shuffling the data
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    LLR = LogisticRegression(random_state=42, penalty='l2', solver='lbfgs', verbose=0, n_jobs=1)
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('LLR', LLR)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
"""code reference from the mne documentation

"""

#this is the Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB


for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    
    #shuffling the data
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    GNB = GaussianNB()
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('GNB', GNB)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
"""code reference from the mne documentation

"""

#this is the Complement Naive Bayes
from sklearn.naive_bayes import ComplementNB


for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    
    #shuffling the data
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    CNB = ComplementNB()
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('CNB', CNB)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
"""code reference from the mne documentation

"""

#this is the Bernoulli Naive Bayes
from sklearn.naive_bayes import BernoulliNB


for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    
    #shuffling the data
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    BNB = BernoulliNB()
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('BNB', BNB)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
"""code reference from the mne documentation

"""

#this is the Categorical Naive Bayes
from sklearn.naive_bayes import CategoricalNB


for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    
    #shuffling the data
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    catNB = CategoricalNB()
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('catNB', catNB)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)

In [ ]:
"""code reference from the mne documentation

"""

#this is the Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB


for f in fp:
    raw = creatRawArray(fp[f])
    events, labels = creatEventsArray(fp[f])
    
    # strip channel names of "." characters
#     raw.rename_channels(lambda x: x.strip('.'))
    
#     print(raw.info)
#     print(raw.ch_names)
#     #adding montage
#     montage = make_standard_montage('standard_1020')
#     raw.set_montage(montage)
    
    
    # Apply band-pass filter
    raw.filter(low_freq, high_freq, fir_design='firwin', skip_by_annotation='edge')
    
#     picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,
#                    exclude='bads')

    # event_train = eventsTrain(fp[f])
    epochs = mne.Epochs(raw, events=events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=None, preload=True,
                        verbose=False)#, picks=picks)

    epochs_train = epochs.copy().crop(tmin=1., tmax=2.)
    labels = epochs.events[:, -1] - 2

    # Define a monte-carlo cross-validation generator (reduce variance):
    scores = []
    epochs_data = epochs.get_data()
    epochs_data_train = epochs_train.get_data()
    
    #shuffling the data
    cv = ShuffleSplit(5, test_size=0.2, random_state=42)
    cv_split = cv.split(epochs_data_train)

    # Assemble a classifier
    MNB = MultinomialNB()
    csp = CSP(n_components=len(epochs.ch_names), reg=None, log=True, norm_trace=False)

    # Use scikit-learn Pipeline with cross_val_score function
    clf = Pipeline([('CSP', csp), ('MNB', MNB)])
    scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=1)

    # Printing the results
    class_balance = np.mean(labels == labels[0])
    class_balance = max(class_balance, 1. - class_balance)
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

    # plot CSP patterns estimated on full data for visualization
#     csp.fit_transform(epochs_data, labels)

#     csp.plot_patterns(epochs.info, ch_type='eeg', units='Patterns (AU)', size=1.5)